In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

이미지 크롤링
- 네이버 웹툰 썸네일 크롤링

In [ ]:
url = 'https://comic.naver.com/webtoon/weekday'

img_srcs = []

resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')

thumb_tags = soup.select('div.list_area.daily_all div.thumb a img')

for thumb_tag in thumb_tags:
    img_srcs.append(thumb_tag['src'])


In [ ]:
for idx, src in enumerate(img_srcs):
    resp = requests.get(src)

    with open('thumb_image/thumb_img_{:03}.jpg'.format(idx), 'wb') as f:
        f.write(resp.content)

FileNotFoundError: ignored

AttributeError: ignored

현상 - 페이지가 바뀌는데 url이 안바뀐다?
해석 
1. 이 url에 할당된 정보가 아니구나
2. 다른 페이지 정보를 빌려온거구나

Iframe : 다른 페이지 정보를 빌려와서 보여주는 것

빌려온 페이지 정보(iframe 정보)는 해당 url의 resp.content로부터 크롤링할 수 없다.(긴빠이칠 수 없다!)

-->> 해당 url에 할당되어있는 페이지 정보만 크롤링할 수 있다.

### 코스피 200(증권) 크롤링



In [ ]:
from tqdm import tqdm

url = 'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200'

resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
a_tags = soup.select('table.summary tbody')

In [ ]:
dates, prices, diffs, rates, volumes, amounts = [], [], [], [], [], []

In [ ]:
date_tags = soup.select('td.date')
num_tags = soup.select('td.number_1')
diff_tags = soup.select('td.rate_down span.tah.p11')
price_tags = num_tags[::4]
rate_tags = num_tags[1::4]
vol_tags = num_tags[2::4]
amu_tags = num_tags[3::4]

for i in range(6):
    dates.append(date_tags[i].text)
    prices.append(price_tags[i].text)
    diff = diff_tags[i].text.strip()

    rate = rate_tags[i].text.strip()
    rates.append(rate)
    volumes.append(vol_tags[i].text)
    amounts.append(amu_tags[i].text)

dates, prices, diffs, rates, volumes, amounts

(['2022.10.28',
  '2022.10.27',
  '2022.10.26',
  '2022.10.25',
  '2022.10.24',
  '2022.10.21'],
 ['295.06', '298.82', '293.85', '291.58', '291.47', '288.57'],
 [],
 ['-1.26%', '+1.69%', '+0.78%', '+0.04%', '+1.00%', '-0.01%'],
 ['129,003', '176,203', '122,301', '116,351', '116,175', '102,369'],
 ['6,379,576',
  '6,808,554',
  '5,868,014',
  '5,506,858',
  '5,372,980',
  '4,677,515'])

In [ ]:

for i in range(len(rate_tags)):
    if rate_tags[i] < 0:
        img_tags[i] = requests.get('https://ssl.pstatic.net/imgstock/images/images4/ico_down.gif')
    else:
        img_tags[i] = requests.get('https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif')



TypeError: ignored

네이버 웹툰 크롤링 추가
- 제목, 작가, 평점에 "장르별" 탭 추가

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

In [ ]:
url = 'https://comic.naver.com/webtoon/genre'

resp = requests.get(url)
resp

<Response [200]>

In [ ]:
g_urls, g_names = [], []

soup = BeautifulSoup(resp.content, 'lxml')
a_tags = soup.select('ul.spot a')
len(a_tags)

for a_tag in a_tags:
    g_urls.append('https://comic.naver.com'+a_tag['href'])
    g_names.append(a_tag.text)
g_urls, g_names

(['https://comic.naver.com/webtoon/genre?genre=episode',
  'https://comic.naver.com/webtoon/genre?genre=omnibus',
  'https://comic.naver.com/webtoon/genre?genre=story',
  'https://comic.naver.com/webtoon/genre?genre=daily',
  'https://comic.naver.com/webtoon/genre?genre=comic',
  'https://comic.naver.com/webtoon/genre?genre=fantasy',
  'https://comic.naver.com/webtoon/genre?genre=action',
  'https://comic.naver.com/webtoon/genre?genre=drama',
  'https://comic.naver.com/webtoon/genre?genre=pure',
  'https://comic.naver.com/webtoon/genre?genre=sensibility',
  'https://comic.naver.com/webtoon/genre?genre=thrill',
  'https://comic.naver.com/webtoon/genre?genre=historical',
  'https://comic.naver.com/webtoon/genre?genre=sports'],
 ['에피소드',
  '옴니버스',
  '스토리',
  '일상',
  '개그',
  '판타지',
  '액션',
  '드라마',
  '순정',
  '감성',
  '스릴러',
  '무협/사극',
  '스포츠'])

In [ ]:
titles, authors, ratings, genres = [], [], [], []

for i in tqdm(range(len(g_urls))):
    genre, url = g_names[i], g_urls[i]
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.content, 'lxml')
    tit_tags = soup.select('ul.img_list li dl dt a')
    aut_tags = soup.select('ul.img_list li dl dd.desc a')
    rat_tags = soup.select('div.rating_type strong')

    for tit_tag, aut_tag, rat_tag in zip(tit_tags, aut_tags, rat_tags):
        titles.append(tit_tag['title'])
        authors.append(aut_tag.text)
        ratings.append(rat_tag.text)
        genres.append(genre)

dic_toons = {
    "제목" : titles,
    "작가" : authors,
    "평점" : ratings,
    "장르" : genres
} 

df_toons = pd.DataFrame(dic_toons)
df_toons

100%|██████████| 13/13 [00:10<00:00,  1.28it/s]


,제목,작가,평점,장르
0,대학원 탈출일지,요다,9.97,에피소드
1,마루는 강쥐,모죠,9.98,에피소드
2,2022 몰래보는 로맨스,웹툰작가,9.85,에피소드
3,약한영웅,서패스 / 김진석,9.82,에피소드
4,비즈니스 여친,루즌아 / 서해,9.74,에피소드
...,...,...,...,...
4037,군주,박산하,8.50,스포츠
4038,미드나잇 체이서,석재윤,9.60,스포츠
4039,불릿 6미리,김도근,9.89,스포츠
4040,리턴,송래현,9.85,스포츠


### 동적이미지 크롤링

자바스크립트 형태인지? (html(구조)/css(디자인)/JavaScript(기능))

iframe 형태인지?

disable JavaScript (판단 후 url 찾기)

- 바뀌면 JS(원래 하던대로 크롤링)
- 안바뀌면?

<Response [200]>